In [2]:
from orphics import sehgal, maps
import healpy as hp
from pixell import utils, enmap, curvedsky, enplot, wcsutils
import os
import numpy as np

import matplotlib.pyplot as plt
import lmdb
from cosmikyu import datasets, transforms

%matplotlib inline
%load_ext autoreload
%autoreload 2

### !!! Refactored / Not Tested

In [ ]:
sehgal_dir = "/home/dwhan89/scratch/data/sehgal_et_al_sims/cosmo_sim_maps/July2009/inputs/microwaveSky"
output_dir = "/home/dwhan89/scratch/data/sehgal_et_al_sims/cosmo_sim_maps/July2009/output/rotated_lmax10000_20mjycuts"
def output_path(x):
    return os.path.join(output_dir, x)

SEHGAL10 = sehgal.SehgalSky2010(path=sehgal_dir, data_type='healpix')
TEMP_ALM = sehgal.SehgalSky2010(path=output_dir, data_type='alm')
TEMP_ENMAP = sehgal.SehgalSky2010(path=output_dir, data_type='enmap')

rfs = {"kappa": lambda : SEHGAL10.get_kappa(),
       "ksz": lambda : SEHGAL10.get_ksz(148),
       "tsz": lambda :SEHGAL10.get_tsz(148),
       "rad_pts": lambda :SEHGAL10.get_radio(148),
       "ir_pts": lambda :SEHGAL10.get_cib(148)
      }

rfs_alm = {"kappa": lambda x: TEMP_ALM.get_kappa(filename_only=x),
       "ksz": lambda x: TEMP_ALM.get_ksz(148,filename_only=x),
       "tsz": lambda x:TEMP_ALM.get_tsz(148,filename_only=x),
       "rad_pts": lambda x:TEMP_ALM.get_radio(148,filename_only=x),
       "ir_pts": lambda x:TEMP_ALM.get_cib(148,filename_only=x)
      }


lmax = 10000
shape, wcs = enmap.fullsky_geometry(res=0.5*utils.arcmin)

    
for compt_idx in ["kappa", "ksz", "tsz"]:
    print(compt_idx)
    alm_ref = None
    alm_file = rfs_alm[compt_idx](True)
    if not os.path.exists(alm_file):
        print("saving alm")
        if alm_ref is None: alm_ref = hp.map2alm(rfs[compt_idx](), lmax=lmax)
        hp.fitsfunc.write_alm(alm_file, alm_ref, overwrite=True)
    else:
        alm_ref = np.complex128(rfs_alm[compt_idx](False))
    for rot_angle1 in rot_angles1:
        for rot_angle2 in rot_angles2:
            file_path = output_path("148_%s_{}_%s_%s_000.fits")
            file_path = file_path %(compt_idx, "%0.3d"%rot_angle1, "%0.3d"%rot_angle2)
            print(file_path)
            alm_path = file_path.format("alm")
            if not os.path.exists(alm_path):
                print("rotating")
                alm = alm_ref.copy()
                if rot_angle2 != 0:
                    hp.rotate_alm(alm, rot_angle1*utils.degree, rot_angle2*utils.degree, 0, lmax=lmax)
                elif rot_angle2 == 0 and rot_angle1 != 0:
                    continue
                else:
                    pass
                hp.fitsfunc.write_alm(alm_path, alm, overwrite=True)
    
    
## with flux cut
NSIDE = 8192
flux_cut = *1e-3 # flux cut in Jy
for compt_idx in ["ir_pts", "rad_pts"]:
    continue
    file_format = output_path("148_%s_{}_%s_%s_000.fits")
    alm_ref = None
    for rot_angle1 in rot_angles1:
        for rot_angle2 in rot_angles2:
            file_path = file_format %(compt_idx, "%0.3d"%rot_angle1, "%0.3d"%rot_angle2)
            alm_path = file_path.format("alm")
            if not os.path.exists(alm_path):
                print(alm_path)
                if rot_angle1 == 0 and rot_angle2 == 0:
                    print("making flux cuts")
                    hpmap = rfs[compt_idx]()
                    area_strad = hp.pixelfunc.nside2pixarea(NSIDE)
                    final_cut = flux_cut/area_strad*sehgal.jysr2thermo(148)
                    loc = np.where(np.abs(hpmap) >= final_cut)
                    hpmap[loc] = 0.
                    alm = hp.map2alm(hpmap, lmax=lmax)
                    hp.fitsfunc.write_alm(alm_path, alm, overwrite=True)
                    continue
                    del hpmap
                elif alm_ref is None:
                    alm_ref_file = file_format %(compt_idx, "%0.3d"%0, "%0.3d"%0)
                    alm_ref_file = alm_ref_file.format("alm")
                    alm_ref = hp.read_alm(alm_ref_file, hdu=(1))
                
                print("rotating")
                alm = alm_ref.copy()
                if rot_angle2 != 0:
                    hp.rotate_alm(alm, rot_angle1*utils.degree, rot_angle2*utils.degree, 0, lmax=lmax)
                elif rot_angle2 == 0 and rot_angle1 != 0:
                    continue
                else:
                    pass
                hp.fitsfunc.write_alm(alm_path, alm, overwrite=True)